# Mosaic

Exploration of mosaicking.

# Imports

In [ ]:
# Native python
import copy
import os

In [ ]:
# External
import cv2
import numpy as np
from osgeo import gdal
import pandas as pd
import sklearn.model_selection
import tqdm
import pyproj

In [ ]:
# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_style('white')

In [ ]:
# Custom scripts
from nitelite_mapmaker import mapmaker, georeference

# Settings

In [ ]:
settings = dict(
    # Data architecture
    flight_name = '220513-FH135',
    data_dir = '/Users/Shared/data/nitelite',
    google_drive_dir = '/Users/zhafensaavedra/Google Drive/Shared drives/NITELite/Data & Analysis',
    flight_subdir = 'Old NITELite Flights/220513-FH135',
    reffed_subdir = 'QGIS FH135/FH135 Main Project/Main Geo Files',
    img_log_filename = 'image.log',
    imu_log_filename = 'OBC/PresIMULog.csv',
    gps_log_filename = 'OBC/GPSLog.csv',
      
    # Choices for what to process
    camera_num = 1,
    test_size = 0.2,
    overwrite_coords_for_reffed = True,
    
    # Data filter choices
    gyro_mag_cut = 0.5, # Corresponds to ~84th percentile
    # percent_for_landed = 95.,
    percent_for_cruising = 85.,
    # mult_of_std_for_steady = 2.,
    # rolling_window_in_min = 1.,
    
    # Mosaicking choices
    allotted_memory = 2., # In GB
    n_tiles_guess = 16,
)

# Set Up

## Settings Parsing

In [ ]:
# The camera has an according long number
settings['camera_long_num'] = settings['camera_num'] + 23085686

In [ ]:
# Data architecture processing
settings['image_dir'] = os.path.join(
    settings['data_dir'],
    'images',
    settings['flight_name'],
    str(settings['camera_long_num'])
)
settings['metadata_dir'] = os.path.join(
    settings['google_drive_dir'],
    settings['flight_subdir'],
    'data',
)
settings['referenced_dir'] = os.path.join(
    settings['google_drive_dir'],
    settings['reffed_subdir'],
)

## Object Creation and Preprocessing

In [ ]:
# Create the main mapmaker object
mm = mapmaker.Mapmaker(
    image_dir=settings['image_dir'],
    img_log_fp=os.path.join(settings['metadata_dir'], settings['img_log_filename']),
    imu_log_fp=os.path.join(settings['metadata_dir'], settings['imu_log_filename']),
    gps_log_fp=os.path.join(settings['metadata_dir'], settings['gps_log_filename']),
    referenced_dir=settings['referenced_dir']
)

In [ ]:
# General metadata loading
mm.prep()

In [ ]:
mm.flight.update_metadata_with_cart_bounds()

In [ ]:
metadata = mm.flight.metadata

In [ ]:
# Color palette
palette = sns.color_palette('deep')

# Exploration

## Determine How Many Images We Can Load

In [ ]:
def get_size_in_GB(fp):
    try:
        return os.path.getsize(fp) / 1024**3
    except FileNotFoundError:
        return np.nan

In [ ]:
# For raw data
filesizes = metadata['filepath'].apply(get_size_in_GB)
median_filesize = np.nanmedian(filesizes)
n_files_in_memory = int(settings['allotted_memory'] // median_filesize)
print(f'Number of raw files to hold in memory: {n_files_in_memory}')

In [ ]:
# For manually-georeferenced data
man_filesizes = metadata.loc[metadata['manually_referenced_fp'].notna(), 'manually_referenced_fp'].apply(get_size_in_GB)
man_median_filesize = np.nanmedian(man_filesizes)
n_man_files_in_memory = int(settings['allotted_memory'] // man_median_filesize)
print(f'Number of raw manually-referenced files to hold in memory: {n_man_files_in_memory}')

## Determine What Images Are Valid

In [ ]:
metadata['valid'] = True

### Cruise Altitude

In [ ]:
h_max = metadata['mAltitude'].max()
h_min = metadata['mAltitude'].min()
h_diff = h_max - h_min

h_cruising = h_min + settings['percent_for_cruising'] / 100. * h_diff

In [ ]:
fig = plt.figure()
ax = plt.gca()

ax.scatter(
    metadata['timestamp'],
    metadata['mAltitude'],
)

ax.axhline(h_cruising)

In [ ]:
metadata.loc[metadata['mAltitude']<h_cruising, 'valid'] = False

### Movement

In [ ]:
# Magnitude of Gyro
metadata['imuGyroMag'] = np.sqrt((metadata[['imuGyroX','imuGyroY','imuGyroZ']]**2.).sum(axis='columns'))

In [ ]:
# Fancy method for movement


# # Select cruise data
# cruise_data = metadata.loc[metadata['flight_phase'] == 'cruise']
# cruise_data = cruise_data.set_index('timestamp')

# # Get rolling deviation
# cruise_rolling = cruise_data.rolling(window=pd.Timedelta(settings['rolling_window_in_min'], 'min'))
# cruise_rolling_std = cruise_rolling.std(numeric_only=True)

# # Identify and store steady data
# cruise_data.loc[:,'is_steady'] = cruise_rolling_std['imuGyroMag'] < settings['mult_of_std_for_steady'] * np.nanmedian(cruise_rolling_std['imuGyroMag'])
# cruise_rolling_std.loc[:,'is_steady'] = cruise_data['is_steady']
# metadata['is_steady'] = False
# metadata.loc[metadata['flight_phase'] == 'cruise','is_steady'] = cruise_data['is_steady'].values



In [ ]:
# Must not be moving too fast
metadata.loc[metadata['imuGyroMag'] > settings['gyro_mag_cut'], 'valid'] = False

### Camera Number

In [ ]:
metadata.loc[metadata['camera_num'] != settings['camera_num'], 'valid'] = False

### Is a Manually-referenced Image

In [ ]:
metadata.loc[~metadata['manually_referenced_fp'].notna(), 'valid'] = False

### Select Data

In [ ]:
selected = metadata.loc[metadata['valid']].copy()
selected.sort_values('timestamp', inplace=True)

# Data split
Only train and test. No validation for right now, but maybe later. Final sample will be using manually referenced ones for validation maybe?

In [ ]:
selected_train, selected_test = sklearn.model_selection.train_test_split(selected, test_size=settings['test_size'])

# Mosaic Creation

## Order of Iteration

In [ ]:
# Determine order for sorting
selected_train.sort_values('timestamp', inplace=True)
indices = selected_train.index

In [ ]:
# Colors
norm = matplotlib.colors.Normalize(vmin=0, vmax=len(indices) - 1)
cmap = sns.color_palette('viridis', as_cmap=True)

In [ ]:
# Plot images, colored by order of overlay
fig = plt.figure(figsize=(15,15))
ax = plt.gca()

# Plot centers
ax.scatter(
    selected_train.loc[indices, 'img_x_center'],
    selected_train.loc[indices, 'img_y_center'],
    color=cmap(norm(np.arange(len(indices)))),
)

# Plot bounds
for i, ind in enumerate(indices):
    row_i = selected_train.loc[ind]

    rect = patches.Rectangle(
        (row_i['img_x_min'], row_i['img_y_min']),
        row_i['img_width'],
        row_i['img_height'],
        linewidth=3,
        facecolor='none',
        edgecolor=cmap(norm(i))
    )
    ax.add_patch(rect)
    
    ax.annotate(
        text=i,
        xy=(row_i['img_x_min'], row_i['img_y_min']),
        xycoords='data',
        xytext=(2, 2),
        textcoords='offset points',
        ha='left',
        va='bottom',
        size = 20,
    )

ax.set_aspect('equal')

## First Few Images

In [ ]:
def plot_imgs(imgs, inds):
    
    subplot_mosaic = [[_, ] for _ in np.arange(len(imgs))]
    n_row = len(subplot_mosaic)
    n_col = len(subplot_mosaic[0])
    
    fig = plt.figure(figsize=(10*n_col, 10*n_row))
    ax_dict = fig.subplot_mosaic(subplot_mosaic)
    
    for i in np.arange(len(imgs)):
        ax = ax_dict[i]
        
        ax.imshow(imgs[i])
    
    for ax_key, ax in ax_dict.items():
        ax.set_aspect('equal')

In [ ]:
# Starting image
ind0 = selected_train.index[0]
obs0 = mm.flight.get_referenced_observation(ind0)

In [ ]:
# First image to incorporate
ind1 = indices[1]
obs1 = mm.flight.get_referenced_observation(ind1)

In [ ]:
## Set expectations for first merging of images
fig = plt.figure(figsize=(20, 20))
ax = plt.gca()

obs0.show(ax=ax, crs='cartesian')
rect = patches.Rectangle(
    (obs0.metadata['img_x_min'], obs0.metadata['img_y_min']),
    obs0.metadata['img_width'],
    obs0.metadata['img_height'],
    linewidth=3,
    facecolor='none',
    edgecolor=palette[0],
    zorder = 999,
)
ax.add_patch(rect)

obs1.show(ax=ax, crs='cartesian')
rect = patches.Rectangle(
    (obs1.metadata['img_x_min'], obs1.metadata['img_y_min']),
    obs1.metadata['img_width'],
    obs1.metadata['img_height'],
    linewidth=3,
    facecolor='none',
    edgecolor=palette[1],
    zorder = 999,
)
ax.add_patch(rect)

ax.set_aspect('equal')

In [ ]:
# Create OpenCV objects
orb = cv2.ORB_create()
bf = cv2.BFMatcher_create(cv2.NORM_HAMMING)

In [ ]:
def incorporate_image(index, dst_img):
    
    row_i = selected_train.loc[index]
    
    img = load_man_img(row_i['manually_referenced_fp'])

    # Detect keypoints in un-referenced
    kp, des = orb.detectAndCompute(img, None)
    dst_kp, dst_des = orb.detectAndCompute(dst_img, None)
    
    # Perform match
    matches = bf.match(des, dst_des)
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)

    # Points for the transform
    src_pts = np.array([kp[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
    dst_pts = np.array([dst_kp[m.trainIdx].pt for m in matches]).reshape(-1,1,2)

    # Get the transform
    M, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.)

    # Corners for image
    img_height, img_width = img.shape[:2]
    corners = np.float32([[0, 0], [0, img_height], [img_width, img_height], [img_width, 0]])
    transformed_corners = cv2.perspectiveTransform(corners.reshape(-1, 1, 2), M)
    
    # Corners for the destination image
    dst_height, dst_width = dst_img.shape[:2]
    dst_corners = np.float32([[0, 0], [0, dst_height], [dst_width, dst_height], [dst_width, 0]])
    
    # Get dimensions of combined image
    all_corners = np.concatenate([transformed_corners.reshape(-1, 2), dst_corners])
    x_min, y_min = all_corners.min(axis=0).astype('int')
    x_max, y_max = all_corners.max(axis=0).astype('int')
    width = x_max - x_min
    height = y_max - y_min
    
    # Translation matrix to shift the transformed image within the new bounds
    translation_matrix = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]]).astype(float)

    # Update the homography matrix to include the translation
    new_M = np.dot(translation_matrix, M)

    # Warp the image being fit
    warped_img = cv2.warpPerspective(img, new_M, (width, height))

    # Translate the dst image
    translated_dst_img = cv2.warpPerspective(dst_img, translation_matrix, (width, height))

    # Make masks for blending. To start we'll want to just overlay images. We can average later.
    # Overlaying means we only want to add warped image where the translated image does not exist
    dst_img_exists = (dst_img.sum(axis=2) > 0).astype(dst_img.dtype)
    translated_dst_img_exists = cv2.warpPerspective(dst_img_exists, translation_matrix, (width, height))

    # Overlay
    include_warped_img = ~(translated_dst_img_exists.astype(bool))
    blended_img = copy.copy(translated_dst_img)
    blended_img[include_warped_img] = warped_img[include_warped_img]
    
    # The blended image now because the destination image
    dst_img = copy.copy(blended_img)
    
    return dst_img, (x_min, x_max, y_min, y_max)

In [ ]:
# First incorporated image

dst_img1, extent1 = incorporate_image(ind1, dst_img)

In [ ]:
plt.imshow(dst_img1, extent=extent1)

## Full Loop

In [ ]:
# Loop through
warped_imgs = []
for i, index in enumerate(tqdm.tqdm(indices)):
        
   

In [ ]:
# Show results
fig = plt.figure()
ax = plt.gca()

ax.imshow(dst_img)

In [ ]:
subplot_mosaic = [[i, ] for i in range(len(warped_imgs))]
fig = plt.figure(figsize=(20, 10*len(subplot_mosaic)))
ax_dict = fig.subplot_mosaic(subplot_mosaic)d

for i in range(len(warped_imgs)):
    ax = ax_dict[i]
    
    ax.imshow(warped_imgs[i])

# Old

Assumed we were going to use the Stitcher class

## Make Tiles

### Determine Tilesize

In [ ]:
def get_regulated_bins(xs, ys, max_count):
    '''Bins that limit the count to max_count
    '''

    x_bins = int(np.sqrt(settings['n_tiles_guess']))
    y_bins = x_bins

    # Refine until we can hold all in memory
    while True:

        # Initial guess for tiling
        hist2d, x_edges, y_edges = np.histogram2d(
            xs,
            ys,
            (x_bins, y_bins),
        )
        hist_max = hist2d.max()

        if hist_max < max_count:
            break

        # Determine tile size based on max density and number of files allowed in memory
        max_surface_density = hist_max / (x_edges[1] - x_edges[0]) / (y_edges[1] - y_edges[0])
        tile_area = n_files_in_memory / max_surface_density
        tile_length = np.sqrt(tile_area)
        x_bins = np.arange(xs.min(), xs.max() + tile_length, tile_length)
        y_bins = np.arange(ys.min(), ys.max() + tile_length, tile_length)
        
    return x_bins, y_bins

In [ ]:
# Convert to get sensor coords
selected['sensor_x'], selected['sensor_y'] = latlon_to_cart.transform(selected['GPSLat'], selected['GPSLong'])
x_bins, y_bins = get_regulated_bins(selected['sensor_x'], selected['sensor_y'], n_files_in_memory)

In [ ]:
fig = plt.figure(figsize=(16,8))
ax_dict = fig.subplot_mosaic([['scatter', 'hist_guess', 'hist']])

ax = ax_dict['scatter']
ax.scatter(
    selected['sensor_x'],
    selected['sensor_y'],
)

ax = ax_dict['hist_guess']
hist2d_guess, x_edges, y_edges, img_view = ax.hist2d(
    selected['sensor_x'],
    selected['sensor_y'],
    (int(np.sqrt(settings['n_tiles_guess'])), int(np.sqrt(settings['n_tiles_guess']))),
)
plt.colorbar(img_view, ax=ax)

ax = ax_dict['hist']
hist2d, x_edges, y_edges, img_view = ax.hist2d(
    selected['sensor_x'],
    selected['sensor_y'],
    (x_bins, y_bins),
)
plt.colorbar(img_view, ax=ax)

for ax_key, ax in ax_dict.items():
    ax.set_aspect('equal')

## Mosaic Creation

## Start w/ Georeferenced Data

In [ ]:
referenced = selected.loc[selected['manually_referenced_fp'].notna()]

## Nested Grids

In [ ]:
x_bins, y_bins = get_regulated_bins(referenced['sensor_x'], referenced['sensor_y'], n_man_files_in_memory)

In [ ]:
fig = plt.figure()
ax = plt.gca()

counts, x_bins, y_bins, hist_img = ax.hist2d(
    referenced['sensor_x'],
    referenced['sensor_y'],
    (x_bins, y_bins),
)

ax.set_aspect('equal')
plt.colorbar(hist_img, ax=ax)

### Stitching for an Single Bin

In [ ]:
# Upper right bin
i = len(x_bins) - 2
j = 0

In [ ]:
# Set the border width here while experimenting
border_width = 0.5
dx = x_bins[1] - x_bins[0]
dy = y_bins[1] = y_bins[0]
in_x_bounds = (
    (x_bins[i] - border_width * dx <= referenced['sensor_x'])
    & (referenced['sensor_x'] <= x_bins[i+1] + border_width * dx)
)
in_y_bounds = (
    (y_bins[i] - border_width * dy <= referenced['sensor_y'])
    & (referenced['sensor_y'] <= y_bins[i+1] + border_width * dy)
)
bounded = referenced.loc[in_x_bounds & in_y_bounds]

In [ ]:
# Load images
def load_man_img(fp):
    
    # Load the manually-referenced image
    man_img = cv2.imread(fp, cv2.IMREAD_UNCHANGED)
    man_img = man_img[:, :, ::-1] / 2**16 # Formatting
    
    man_img_int = cv2.normalize(man_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    return man_img_int
cell_imgs = [load_man_img(fp) for fp in bounded['manually_referenced_fp']]

In [ ]:
# Stitch
stitcher = cv2.Stitcher_create(1)
status, mosaick = stitcher.stitch(cell_imgs)

In [ ]:
# Inspect
fig = plt.figure(figsize=(10,10))
ax = plt.gca()

ax.imshow(
    mosaick
)

Uh oh, defects abound. We'll need to dig into the stitcher...

## Image Pyramids

If we reduce the resolution sufficiently, we can stitch all the images at once, and then iterate from there. Let's calculate what the resolution would be of the new images.

In [ ]:
new_resolution = tuple(np.ceil(np.array(mm.flight.img_shape) / np.sqrt(n_files_in_memory)).astype(np.int32))
print(f'Old resolution = {mm.flight.img_shape}')
print(f'New resolution = {new_resolution}')

It's not clear that downsampling the images this much would work.

## Retrieve Coordinates for Manually-Referenced Images

In [ ]:
def get_bounds(self, dst_crs=latlon_crs):
    
    # Get the coordinates
    x_min, px_width, x_rot, y_max, y_rot, px_height = self.GetGeoTransform()
    x_max = x_min + px_width * dataset.RasterXSize
    y_min = y_max + px_height * dataset.RasterYSize
    
    man_crs = pyproj.CRS(self.GetProjection())
    man_to_dst = pyproj.Transformer.from_crs(man_crs, dst_crs, always_xy=True)
    
    x_bounds, y_bounds = man_to_dst.transform([x_min, x_max], [y_min, y_max])
    
    return (x_bounds, y_bounds)

gdal.Dataset.get_bounds = get_bounds

In [ ]:
man_coords_fp = os.path.join(settings['referenced_dir'], 'img_coordinates_{}.csv'.format(cart_crs.srs.replace(':', '')))

In [ ]:
if os.path.exists(man_coords_fp) and not settings['overwrite_coords_for_reffed']:
    manually_referenced = pd.read_csv(man_coords_fp, index_col=0)
else:
    
    
    calculated = {
        'man_img_x_min': [],
        'man_img_x_max': [],
        'man_img_y_min': [],
        'man_img_y_max': [],
    }

    man_fps = metadata.loc[metadata['manually_referenced_fp'].notna(), 'manually_referenced_fp']
    for i in tqdm.tqdm(range(len(man_fps))):

        man_fp = man_fps.iloc[i]
        ind = man_fps.index[i]
        row = metadata.loc[ind]

        dataset = gdal.Open(man_fp)
        x_lim_man, y_lim_man = dataset.get_bounds(cart_crs)

        calculated['man_img_x_min'].append(x_lim_man[0])
        calculated['man_img_x_max'].append(x_lim_man[1])
        calculated['man_img_y_min'].append(y_lim_man[0])
        calculated['man_img_y_max'].append(y_lim_man[1])

    # Make a dataframe
    manually_referenced = pd.DataFrame(calculated)
    manually_referenced.index = man_fps.index
    manually_referenced['filename'] = metadata['manually_referenced_fp'].str.split('/').str[-1]

    # Save
    manually_referenced.to_csv(man_coords_fp)

In [ ]:
# Manual image centers
manually_referenced['man_x_center'] = 0.5 * (manually_referenced['man_img_x_min'] + manually_referenced['man_img_x_max'])
manually_referenced['man_y_center'] = 0.5 * (manually_referenced['man_img_y_min'] + manually_referenced['man_img_y_max'])

In [ ]:
# Manual image dimensions
manually_referenced['man_img_width'] = manually_referenced['man_img_x_max'] - manually_referenced['man_img_x_min']
manually_referenced['man_img_height'] = manually_referenced['man_img_y_max'] - manually_referenced['man_img_y_min']
manually_referenced['man_img_hypotenuse'] = np.sqrt(manually_referenced['man_img_width']**2. + manually_referenced['man_img_height']**2.)

In [ ]:
metadata = metadata.join(manually_referenced, rsuffix='_man')